In [ ]:
import re
import requests
import numpy as np
import pandas as pd
import plotly.express as px
from bs4 import BeautifulSoup
from functools import wraps

In [ ]:
# Utils

def get_response(url):
    user_agent_header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                       AppleWebKit/605.1.15 (KHTML, like Gecko) \
                       Chrome/100.0.4896.127 Safari/605.1.15 Firefox/100.0" 
        }

    session = requests.Session()
    response = session.get(url, headers=user_agent_header)

    return response


def parse_quotes(data):
    return float(re.search("[0-9\.\-]+", data.get_text()).group())

In [ ]:
class SingleBase:
    def __init__(self, ticker, region="usa"):
        self._ticker = ticker.upper()
        self._region = region
        self._rowspan = 11


    def _get_short_name(self):
        url = f"http://performance.morningstar.com/stock/performance-return.action?t={self._ticker}&region={self._region}&culture=en-US"

        response = get_response(url).text
        soup = BeautifulSoup(response)

        short_name = soup.find("h1").get_text()

        return short_name


    def _get_quote_data(self):
        url = f"http://performance.morningstar.com/perform/Performance/stock/quote-data-strip.action?&t={self._ticker}&region={self._region}&culture=en-US"

        response = get_response(url).text
        soup = BeautifulSoup(response)

        data = soup.find_all("span", attrs={"class": "data_big"})
        last_price = parse_quotes(data[0])
        daily_change = parse_quotes(data[1])
        daily_pct_change = parse_quotes(data[2])

        quote_data = {
                    "Last Price": last_price,
                    "Daily Change": daily_change,
                    "Daily Change (%)": daily_pct_change
                    }

        return quote_data


    def _get_performance(self, data):
        if data == "history":
            url = f"http://performance.morningstar.com/perform/Performance/stock/performance-history.action?&t={self._ticker}&region={self._region}&culture=en-US"
        elif data == "returns":
            url = f"http://performance.morningstar.com/perform/Performance/stock/trailing-total-returns.action?&t={self._ticker}&region={self._region}&culture=en-US"

        df = pd.read_html(url, header=0, index_col=0)[0]
        df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

        return df


    def _get_price_history(self, period, frequency):
        url = f"http://performance.morningstar.com/perform/Performance/stock/price-history.action?&t={self._ticker}&region={self._region}&culture=en-US&pd={period}&freq={frequency}"

        df = pd.read_html(url, header=0, index_col=0, keep_default_na=False, parse_dates=True)[0]
        df["Volume"] = df["Volume"].apply(lambda x: pd.to_numeric(x.replace("Mil", ""), errors="coerce") * 10**6)
        df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))
        df.rename_axis(index=None, inplace=True)

        df.dropna(axis=0, how="all", inplace=True)

        return df


    def _visualize_price_history(self, data):
        chart = px.area(x=data.index, y=data["Close"])

        axes_layout = {
                       "gridcolor": "rgb(240, 240, 240)",
                       "showspikes": True,
                       "spikemode": "across",
                       "spikesnap": "cursor",
                       "spikedash": "dash",
                       "spikethickness": 0.5
                      }

        chart.update_xaxes(
            axes_layout,
            title_text="Date",
            rangeselector={
                "buttons": [
                    {"count": 5, "label": "5D", "step": "day", "stepmode": "backward"},
                    {"count": 1, "label": "1M", "step": "month", "stepmode": "backward"},
                    {"count": 3, "label": "3M", "step": "month", "stepmode": "backward"},
                    {"count": 6, "label": "6M", "step": "month", "stepmode": "backward"},
                    {"count": 1, "label": "YTD", "step": "year", "stepmode": "todate"},
                    {"count": 1, "label": "1Y", "step": "year", "stepmode": "backward"},
                    {"count": 3, "label": "3Y", "step": "year", "stepmode": "backward"},
                    {"count": 5, "label": "5Y", "step": "year", "stepmode": "backward"},
                    {"count": 10, "label": "10Y", "step": "year", "stepmode": "backward"},
                    {"label": "MAX", "step": "all"}]})

        chart.update_yaxes(axes_layout, title_text="Price", tickprefix="$")
        
        chart.update_layout(
            plot_bgcolor="rgb(255, 255, 255)",
            hovermode="x",
            spikedistance=-1,
            hoverdistance=-1,
            showlegend=False,
            title={
                "text": f"{self._ticker} Stock Price History",
                "y": 0.97,
                "x": 0.5,
                "xanchor": "center",
                "yanchor": "top"})

        chart.update_traces(hovertemplate="Date: %{x}<br>Price: %{y}")

        chart.show();


    def _get_annual_dividends(self):
        url = f"http://performance.morningstar.com/perform/Performance/stock/annual-dividends.action?&t={self._ticker}&region={self._region}&culture=en-US"

        df = pd.read_html(url, header=0, index_col=0)[0]
        df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))
        df = df.loc[["Dividend Amount", "Year-end Yield %"]]

        return df


    def _get_dividend_history(self):
        url = f"http://performance.morningstar.com/perform/Performance/stock/dividend-history.action?&t={self._ticker}&region={self._region}&culture=en-US"

        response = get_response(url).text
        soup = BeautifulSoup(response)

        th_str = soup.find_all("th", attrs={"class": "str"})
        cols = [col.get_text() for col in th_str]

        td = soup.find_all("td")
        rows = [row.get_text() for row in td]
        rows = [rows[i: i+len(cols)] for i in range(0, len(rows), len(cols))]

        df = pd.DataFrame(data=rows, columns=cols)
        cols_to_dt = ["Ex-Dividend Date", "Declaration Date", "Record Date", "Payable Date"]
        df[cols_to_dt] = df[cols_to_dt].apply(pd.to_datetime)
        df["Amount"] = df["Amount"].apply(lambda x: float(x.replace("$", "")))

        return df


    def _get_key_finanacial_ratios(self):
        url = f"http://financials.morningstar.com/finan/financials/getFinancePart.html?&t={self._ticker}&region={self._region}&culture=en-US"

        response = get_response(url).json()
        soup = BeautifulSoup(response["componentData"])

        th_col = soup.find_all("th", attrs={"scope": "col"})
        cols = [col.get_text() for col in th_col]

        th_row = soup.find_all("th", attrs={"scope": "row"})
        lbls = [lbl.get_text().replace(" *", "").replace("\xa0", " ") for lbl in th_row][1:]

        td = soup.find_all("td")
        rows = [row.get_text().replace(",", "") for row in td if row.get_text()][:-1]
        rows = [rows[i: i+len(cols)] for i in range(0, len(rows), len(cols))]

        df = pd.DataFrame(data=rows, columns=cols, index=lbls)
        df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

        return df


    def _total_key_ratios_data(function):

        @wraps(function)
        def wrapper(self, *args, **kwargs):
            url = f"http://financials.morningstar.com/finan/financials/getKeyStatPart.html?&t={self._ticker}&region={self._region}&culture=en-US"

            response = get_response(url).json()
            soup = BeautifulSoup(response["componentData"])

            th_col = soup.find_all("th", attrs={"scope": "col"})
            cols = [col.get_text() for col in th_col]

            th_row = soup.find_all("th", attrs={"scope": "row"})
            lbls = [lbl.get_text().replace(" *", "").replace("\xa0", " ") for lbl in th_row]

            td = soup.find_all("td")
            rows = [row.get_text().replace(",", "") for row in td if row.get_text()]
            rows = [rows[i: i+self._rowspan] for i in range(0, len(rows), self._rowspan)]

            row_start, row_end, col_start, col_end, lbl_start, lbl_end, flag = function(self, *args, **kwargs)

            if flag:
                [rows.insert(i, ["—"] * self._rowspan) for i in range(row_start, row_end, 5)]
                row_end += 4
                lbl_end += 4

            df = pd.DataFrame(data=rows[row_start:row_end], columns=cols[col_start:col_end], index=lbls[lbl_start:lbl_end])
            df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

            return df

        return wrapper


    @_total_key_ratios_data
    def _get_margins(self):
        data = [0, 9, 1, 12, 0, 9, False]
        return data


    @_total_key_ratios_data
    def _get_profitability(self):
        data = [9, 17, 13, 24, 9, 17, False]
        return data


    @_total_key_ratios_data
    def _get_growth(self):
        data = [17, 33, 24, 35, 17, 33, True]
        return data


    @_total_key_ratios_data
    def _get_cash_flow_ratios(self):
        data = [33, 38, 36, 47, 37, 42, False]
        return data


    @_total_key_ratios_data
    def _get_balance_sheet_items(self):
        data = [38, 58, 48, 59, 42, 62, False]
        return data


    @_total_key_ratios_data
    def _get_liquidity(self):
        data = [58, 62, 60, 71, 62, 66, False]
        return data


    @_total_key_ratios_data
    def _get_efficiency(self):
        data = [62, 70, 72, 83, 66, 74, False]
        return data


    def _get_financials(self, statement):
        url = f"http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t={self._ticker}&region={self._region}&culture=en-US&reportType={statement}"

        response = get_response(url).json()
        soup = BeautifulSoup(response["result"])

        cols = soup.find_all("div", attrs={"class": "year"})
        cols = [col.get_text() for col in cols]

        lbls = soup.find_all("div", attrs={"class": "lbl"})[1:]
        alts = soup.find_all("div", title=True, attrs={"class": "lbl"})

        lbls = [lbl.get_text() for lbl in lbls]
        lbls = list(filter(lambda x : x != "\xa0", lbls))

        for i, lbl in enumerate(lbls):
            for j, alt in enumerate(alts):
                if lbl == alt.get_text():
                    lbls[i] = alts[j]["title"]

        rows = soup.find_all("div", attrs={"class": "pos"})
        rows = [row.get_text().replace(",", "").replace("(", "-").replace(")", "") for row in rows]
        rows = [rows[i: i+len(cols)] for i in range(0, len(rows), len(cols))]

        df = pd.DataFrame(data=rows, columns=cols, index=lbls)
        df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

        return df


    def _get_valuation(self, table):
        if table == "current-valuation":
            url = f"http://financials.morningstar.com/valuate/current-valuation-list.action?&t={self._ticker}&region={self._region}&culture=en-US"
        elif table == "forward-valuation":
            url = f"http://financials.morningstar.com/valuate/forward-valuation-list.action?&t={self._ticker}&region={self._region}&culture=en-US"
        elif table == "forward-comparisons":
            url = f"http://financials.morningstar.com/valuate/forward-comparisons-list.action?&t={self._ticker}&region={self._region}&culture=en-US"

        df = pd.read_html(url, header=0, index_col=0)[0]
        df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

        df.dropna(axis=0, how="all", inplace=True)
        df.dropna(axis=1, how="all", inplace=True)

        return df


    def _get_valuation_history(self):
        url = f"http://financials.morningstar.com/valuate/valuation-history.action?&t={self._ticker}&region={self._region}&culture=en-US&type=price-earnings"

        df = pd.read_html(url, index_col=0)[0]
        df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))
        
        df.columns = self._get_key_finanacial_ratios().columns
        df.rename_axis(index=None, inplace=True)

        return df


    def _get_eight_pillars(self):
        income_statement = self._get_financials("is")

        pe_ratio = self._get_valuation_history().loc[:"Price/Book"].loc[self._ticker][5:10]
        avg_pe_ratio = pe_ratio.mean()

        roic = self._get_profitability().loc["Return on Invested Capital %"][5:10]
        avg_roic = roic.mean()

        revenue = income_statement.loc["Revenue"][:-1]
        revenue_change = revenue[-1] - revenue[0]
        revenue_change = int(revenue_change) if not pd.isna(revenue_change) else revenue_change

        net_income = income_statement.loc["Net income"][:-1]
        net_income_change = net_income[-1] - net_income[0]
        net_income_change = int(net_income_change) if not pd.isna(net_income_change) else net_income_change

        shares_outstanding = income_statement.loc["Weighted average shares outstanding":].loc["Diluted"][:-1]
        shares_outstanding_change = (shares_outstanding[-1] / shares_outstanding[0] - 1) * 100

        free_cash_flow = self._get_financials("cf").loc["Free cash flow"][:-1]
        free_cash_flow_change = free_cash_flow[-1] - free_cash_flow[0]
        free_cash_flow_change = int(free_cash_flow_change) if not pd.isna(free_cash_flow_change) else free_cash_flow_change
        avg_free_cash_flow = free_cash_flow.mean()

        long_term_liabilities = self._get_financials("bs").loc["Total non-current liabilities"][-1]
        ltl_to_fcf = long_term_liabilities / avg_free_cash_flow

        current_price = self._get_quote_data()["Last Price"]
        market_cap = shares_outstanding[-1] * current_price
        price_to_fcf = market_cap / avg_free_cash_flow

        pillar_1 = "✔️" if avg_pe_ratio < 22.5 else "❌"
        pillar_2 = "✔️" if avg_roic > 9 else "❌"
        pillar_3 = "✔️" if revenue_change > 0 else "❌"
        pillar_4 = "✔️" if net_income_change > 0 else "❌"
        pillar_5 = "✔️" if shares_outstanding_change < 0 else "❌"
        pillar_6 = "✔️" if ltl_to_fcf < 5 else "❌"
        pillar_7 = "✔️" if free_cash_flow_change > 0 else "❌"
        pillar_8 = "✔️" if price_to_fcf < 22.5 else "❌"

        print(f"{pillar_1} Pillar #1: 5-Year P/E Ratio < 22.5            = {avg_pe_ratio:.2f}")
        print(f"{pillar_2} Pillar #2: 5-Year ROIC > 9%                   = {avg_roic:.2f}%")
        print(f"{pillar_3} Pillar #3: 5-Year Revenue Growth (Mil)        = {revenue_change}")
        print(f"{pillar_4} Pillar #4: 5-Year Net Income Growth (Mil)     = {net_income_change}")
        print(f"{pillar_5} Pillar #5: 5-Year Shares Outstanding          = {shares_outstanding_change:.2f}%")
        print(f"{pillar_6} Pillar #6: 5 Year LTL to FCF < 5              = {ltl_to_fcf:.2f}")
        print(f"{pillar_7} Pillar #7: 5-Year Free Cash Flow Growth (Mil) = {free_cash_flow_change}")
        print(f"{pillar_8} Pillar #8: 5-Year Price to FCF < 22.5         = {price_to_fcf:.2f}")


    def _get_intrinsic_value(self, to_csv=False):
        company_name = self._get_short_name()
        industry = self._get_performance("history").index[1]
        current_price = self._get_quote_data()["Last Price"]

        free_cash_flow = self._get_financials("cf").loc["Free cash flow"][-1]

        try:
            growth_rates = {}
            forward_free_cash_flow = {}
            npv_free_cash_flow = {}

            estimated_growth_rate = self._get_valuation("forward-comparisons").loc[self._ticker][0] / 100

            for i in range(1, 11):
                growth_rates[i] = growth_rates[i - 1] * (1.0 - GROWTH_DECLINE_RATE) \
                                if i > 1 else estimated_growth_rate

                forward_free_cash_flow[i] = forward_free_cash_flow[i - 1] * (1.0 + growth_rates[i]) \
                                            if i > 1 else free_cash_flow * (1.0 + growth_rates[i])

                npv_free_cash_flow[i] = forward_free_cash_flow[i] / (1.0 + DISCOUNT_RATE) ** i

            shares_outstanding = self._get_key_finanacial_ratios().loc["Shares Mil"][-1]
            market_cap = current_price * shares_outstanding

            balance_sheet = self._get_financials("bs")

            short_term_debt = balance_sheet.loc["Short-term debt"][-1]
            long_term_debt = balance_sheet.loc["Long-term debt"][-1]
            cash_and_equivalents = balance_sheet.loc["Cash and cash equivalents"][-1]
            total_debt = np.nansum([short_term_debt, long_term_debt])
            net_debt = total_debt - cash_and_equivalents

            total_npv_free_cash_flow = sum(npv_free_cash_flow.values())
            terminal_multiple = market_cap / free_cash_flow
            terminal_value = npv_free_cash_flow[10] * terminal_multiple
            equity_value = total_npv_free_cash_flow + terminal_value - net_debt

            intrinsic_value = equity_value / shares_outstanding

            declined_growth_rate = growth_rates[10]

            print(f"Company Ticker                           =  {self._ticker}")
            print(f"Company Name                             =  {company_name}")
            print(f"Industry                                 =  {industry}")
            print("-" * 60)
            print(f"Free Cash Flow (Mil)                     =  {int(free_cash_flow)}")
            print(f"Net Debt (Mil)                           =  {int(net_debt)}")
            print(f"Shares Outstanding (Mil)                 =  {int(shares_outstanding)}")
            print(f"Estimated Growth Rate (+5Y)              =  {round(estimated_growth_rate * 100, 2)}%")
            print(f"Growth Decline Rate                      =  {round(GROWTH_DECLINE_RATE * 100, 2)}%")
            print(f"Declined Estimated Growth Rate (10Y)     =  {round(declined_growth_rate * 100, 2)}%")
            print(f"Discount Rate                            =  {round(DISCOUNT_RATE * 100, 2)}%")
            print(f"Terminal Multiple                        =  {terminal_multiple:.2f}")
            print("-" * 60)
            print(f"Total NPV FCF (Mil)                      =  {total_npv_free_cash_flow:.2f}")
            print(f"Terminal Value (Mil)                     =  {terminal_value:.2f}")
            print(f"Enterprise Value (Mil)                   =  {equity_value:.2f}")
            print("-" * 60)
            print(f"Current Price                            =  {current_price}")
            print(f"Intrinsic Value                          =  {intrinsic_value:.2f}")
        except:
            return None

        if to_csv:
            results[self._ticker] = {
                                    "Company Name": company_name,
                                    "Industry": industry,
                                    "Current Price": current_price,
                                    "Intrinsic Value": round(intrinsic_value, 2)
                                }

            return pd.DataFrame.from_dict(data=results).T.to_csv("dcf-model-results.csv", index=self._ticker)

In [ ]:
class Ticker(SingleBase):
    def __repr__(self):
        return "morningstar.Ticker object <%s>" % self._ticker

    @property
    def short_name(self):
        return self._get_short_name()

    @property
    def quote_data(self):
        return self._get_quote_data()

    @property
    def performance_history(self):
        return self._get_performance("history")

    @property
    def trailing_total_returns(self):
        return self._get_performance("returns")

    def price_history(self, period="5d", frequency="d"):
        """
        :Parameters:
            period : str
                Valid periods: 5d, 1m, 3m, ytd, 1y, 3y, 5y, 10y, max
                Alternatively, you can use any decimal number with the following letters:
                "d", "m", "y" representing days, months and years to get the required period.
                E.g. 3d, 4m, 7y.

            frequency : str
                Valid frequencies: d, w, m, q, a
        """
        return self._get_price_history(period, frequency)
    
    @property
    def chart(self):
        return self._visualize_price_history(self._get_price_history(period="max", frequency="d"))

    @property
    def annual_dividends(self):
        return self._get_annual_dividends()

    @property
    def dividend_history(self):
        return self._get_dividend_history()

    @property
    def key_finanacial_ratios(self):
        return self._get_key_finanacial_ratios()

    @property
    def margins(self):
        return self._get_margins()

    @property
    def profitability(self):
        return self._get_profitability()

    @property
    def growth(self):
        return self._get_growth()

    @property
    def cash_flow_ratios(self):
        return self._get_cash_flow_ratios()

    @property
    def balance_sheet_items(self):
        return self._get_balance_sheet_items()

    @property
    def liquidity(self):
        return self._get_liquidity()

    @property
    def efficiency(self):
        return self._get_efficiency()

    @property
    def income_statement(self):
        return self._get_financials("is")

    @property
    def balance_sheet(self):
        return self._get_financials("bs")

    @property
    def cash_flow(self):
        return self._get_financials("cf")

    @property
    def current_valuation(self):
        return self._get_valuation("current-valuation")

    @property
    def forward_valuation(self):
        return self._get_valuation("forward-valuation")

    @property
    def forward_comparisons(self):
        return self._get_valuation("forward-comparisons")

    @property
    def valuation_history(self):
        return self._get_valuation_history()

    @property
    def eight_pillars(self):
        return self._get_eight_pillars()
    
    @property
    def intrinsic_value(self):
        return self._get_intrinsic_value(to_csv=False)

    @property
    def intrinsic_value_to_csv(self):
        return self._get_intrinsic_value(to_csv=True)

In [ ]:
GROWTH_DECLINE_RATE = 0.1
DISCOUNT_RATE = 0.125

In [ ]:
company = Ticker("aapl")
company.eight_pillars

In [ ]:
# !pip install aiohttp

import asyncio
import aiohttp

import nest_asyncio
nest_asyncio.apply()

In [ ]:
class MultiBase:
    def __init__(self, tickers):
        tickers = tickers if isinstance(tickers, (list, set, tuple)) else tickers

        self._tickers = [ticker.upper() for ticker in tickers]


    def _get_price_history(self, ticker, data):
        df = pd.read_html(data, header=0, index_col=0, parse_dates=True)[0]["Close"]
        df.dropna(inplace=True)
        df.rename(ticker, inplace=True)

        return df


    async def _get_response(self, ticker, session, period, frequency):
        url = f"http://performance.morningstar.com/perform/Performance/stock/price-history.action?&t={ticker}&pd={period}&freq={frequency}"

        async with session.get(url) as response:
            data = await response.text()

            return self._get_price_history(ticker, data)


    async def _gather_tasks(self, period, frequency):
        async with aiohttp.ClientSession() as session:
            return pd.concat(
                await asyncio.gather(*[
                                    asyncio.create_task(self._get_response(ticker, session, period, frequency))
                                    for ticker in self._tickers
                                    ]),
                axis=1
                ).sort_index(ascending=False)

In [ ]:
class Tickers(MultiBase):
    def __repr__(self):
        return "morningstar.Tickers object <%s>" % ", ".join(self._tickers)

    def price_history(self, period="5d", frequency="d"):
        """
        :Parameters:
            period : str
                Valid periods: 5d, 1m, 3m, ytd, 1y, 3y, 5y, 10y, max
                Alternatively, you can use any decimal number with the following letters:
                "d", "m", "y" representing days, months and years to get the required period.
                E.g. 3d, 4m, 7y.

            frequency : str
                Valid frequencies: d, w, m, q, a
        """
        return asyncio.run(self._gather_tasks(period, frequency))

In [ ]:
tickers = [
           "AAPL", "MSFT", "AMZN", "GOOGL", "FB", "INTC", "AMD", "MU", "IBM", "HPQ",
           "COST", "LOW", "TGT", "WMT", "HD", "BBY", "HON", "KO", "PEP", "MCD",
           "XOM", "CVX", "MA", "V", "PYPL", "CAT", "DE", "MMM", "TJX", "WM",
           "UNH", "ABBV", "ABT", "AMGN", "MRK", "PFE", "LLY", "TMO", "BMY", "PG",
           "JNJ", "CL", "NKE"
           ]

companies = Tickers(tickers)
companies.price_history("max")

,AAPL,MSFT,AMZN,GOOGL,FB,INTC,AMD,MU,IBM,HPQ,...,AMGN,MRK,PFE,LLY,TMO,BMY,PG,JNJ,CL,NKE
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-01,148.71,272.42,2433.68,2277.84,188.64,44.11,101.22,73.55,139.43,40.34,...,253.42,91.14,52.37,311.08,556.22,74.96,145.64,177.71,78.20,118.68
2022-05-31,148.84,271.87,2404.19,2275.24,193.64,44.42,101.86,73.84,138.84,38.84,...,256.74,92.03,53.04,313.44,567.57,75.45,147.88,179.53,78.81,118.85
2022-05-30,149.64,273.24,2302.93,2246.33,195.13,44.55,102.26,73.32,139.27,38.75,...,255.26,93.08,53.91,323.48,572.35,76.14,148.72,181.09,78.86,115.99
2022-05-27,149.64,273.24,2302.93,2246.33,195.13,44.55,102.26,73.32,139.27,38.75,...,255.26,93.08,53.91,323.48,572.35,76.14,148.72,181.09,78.86,115.99
2022-05-26,143.78,265.90,2221.55,2155.85,191.63,43.48,98.75,70.60,136.89,36.77,...,253.05,92.31,53.99,313.46,540.11,77.59,146.48,179.46,78.16,112.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-11-15,3.00,29.13,42.60,246.20,NaN,22.32,21.71,14.83,88.94,18.06,...,73.61,42.10,25.13,54.37,43.94,24.50,63.12,66.54,32.16,11.94
2006-11-14,3.04,29.42,41.51,245.21,NaN,21.88,21.34,14.95,89.11,18.46,...,73.14,41.85,24.87,53.85,43.71,24.44,63.20,66.56,32.37,11.84
2006-11-13,3.01,29.35,39.99,240.75,NaN,21.02,21.20,14.21,87.94,18.26,...,72.70,41.44,24.53,54.06,43.62,24.41,63.25,65.94,32.44,11.77
